### https://www.kaggle.com/c/tmdb-box-office-prediction

In [323]:
import pandas as pd 
from sklearn import datasets
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

#def rmsle_score(y, y0):
#    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))

In [324]:
train = pd.read_csv('TMDB_Box_Office/train.csv')
test = pd.read_csv('TMDB_Box_Office/test.csv')

In [322]:
train.head(20)['belongs_to_collection'][11]

"[{'id': 48190, 'name': 'Revenge of the Nerds Collection', 'poster_path': '/qOnoXEdrSnBuS3FMAFRIgyJSM2r.jpg', 'backdrop_path': None}]"

In [173]:
train.shape, test.shape

((3000, 23), (4398, 22))

### belongs_to_collection

In [174]:
for i, e in enumerate(train['belongs_to_collection'][:5]):
    print(i, e)

0 [{'id': 313576, 'name': 'Hot Tub Time Machine Collection', 'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg', 'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg'}]
1 [{'id': 107674, 'name': 'The Princess Diaries Collection', 'poster_path': '/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg', 'backdrop_path': '/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg'}]
2 nan
3 nan
4 nan


In [175]:
#2396 values in this column are empty, 604 contrain information about the collections
(~train['belongs_to_collection'].isna()).value_counts()
#(~train['belongs_to_collection'].isna()).apply(lambda x: np.where(x, 0, 1))
#train['has_collection'] = ~train['belongs_to_collection'].isna().astype()
#train['has_collection'] 
#train['belongs_to_collection'].apply(lambda x: '1' if x is np.NaN else '0')

False    2396
True      604
Name: belongs_to_collection, dtype: int64

In [205]:
#only collection name can be useful. Another possibly useful feature is the fact of belonging to a collection.

train['has_collection'] = train['belongs_to_collection'].apply(lambda x: 1 if x is not np.NaN else 0)
train['has_collection'] 

test['has_collection'] = test['belongs_to_collection'].apply(lambda x: 1 if x is not np.NaN else 0)
test['has_collection'] 





0       1
1       0
2       0
3       0
4       0
       ..
4393    0
4394    1
4395    0
4396    0
4397    0
Name: has_collection, Length: 4398, dtype: int64

In [342]:
def str_to_dict(value):
    #print(value)
    if value is not np.NaN:
        value_ = value[1:-1]
        #print(value_)
        value_ = value_.replace('\'','\"')
        
        print('YYY',value_)
        value_as_dict = json.loads(value_)
        print('XXX',value_as_dict['name'])
        return value_as_dict['name']
    else:
         return np.NaN
    
    #if value is not np.NaN:
     #   value_ = value[1:-1]
        #value_ = value_.replace('\'','\"')
        #value_as_dict = json.loads(value_)
      #  return value
    #else:
     #   return 'missing'
    


In [343]:
train['collection_name']  = train['belongs_to_collection'].apply(lambda x: str_to_dict(x) if x is not np.NaN else 0)
train['collection_name'] 

YYY {"id": 313576, "name": "Hot Tub Time Machine Collection", "poster_path": "/iEhb00TGPucF0b4joM1ieyY026U.jpg", "backdrop_path": "/noeTVcgpBiD48fDjFVic1Vz7ope.jpg"}
XXX Hot Tub Time Machine Collection
YYY {"id": 107674, "name": "The Princess Diaries Collection", "poster_path": "/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg", "backdrop_path": "/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg"}
XXX The Princess Diaries Collection
YYY {"id": 256377, "name": "The Muppet Collection", "poster_path": "/8Ew8EIdFFurMMYjSbWPu1Hl4vLX.jpg", "backdrop_path": "/1AWd3MM90G47mxtD112gRDxSXY9.jpg"}
XXX The Muppet Collection
YYY {"id": 1575, "name": "Rocky Collection", "poster_path": "/mCY5dMkSSFQufGCViI6jNUU6pXq.jpg", "backdrop_path": "/w4h6gjdWPvmu5R9H6zeGDPo1ZuV.jpg"}
XXX Rocky Collection
YYY {"id": 48190, "name": "Revenge of the Nerds Collection", "poster_path": "/qOnoXEdrSnBuS3FMAFRIgyJSM2r.jpg", "backdrop_path": None}


JSONDecodeError: Expecting value: line 1 column 126 (char 125)

In [349]:
s = {"id": 48190, "name": "Revenge of the Nerds Collection", "poster_path": "/qOnoXEdrSnBuS3FMAFRIgyJSM2r.jpg", "backdrop_path": None}
s1 = str({"id": 313576, "name": 'Hot Tub Time Machine Collection', "poster_path": "/iEhb00TGPucF0b4joM1ieyY026U.jpg", "backdrop_path": "/noeTVcgpBiD48fDjFVic1Vz7ope.jpg"})

dd = json.loads(s1)
dd

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [195]:
for i, e in enumerate(train['genres'][:5]):
    print(i, e)
    

0 [{'id': 35, 'name': 'Comedy'}]
1 [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]
2 [{'id': 18, 'name': 'Drama'}]
3 [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}]
4 [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]


In [ ]:


X = df.drop('revenue',axis=1)
y = df.revenue

X_train,X_test,y_train ,y_test = train_test_split(X,y,random_state = 42,train_size = 0.8,test_size=0.2)
X_train,X_test ,y_train ,y_test

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)
scores = cross_val_score(lin_reg, X, y, cv = 5)
    

#scores = cross_val_score(lin_reg,X_train,y_train,cv =10,scoring=rmsle_score)
#scores,scores.mean(),scores.std()

